In [ ]:
import torch
from utils.base_model import BiLSTM_CRF
from utils.utils import prepare, predict, train, evaluate

hparams = {
    'path':'data/data.txt',
    'epochs': 80,
    'batch_size': 100,
    'embedding_dim': 300,
    'hidden_dim': 256,
    'device':'cuda:1',
    'seq_length': 256,
    'save_path':'model_params/base_model'
}

attr_dict, loaders = prepare(hparams)
hparams['vocab_size'] = len(attr_dict['tokenizer'])

In [ ]:
base_model = BiLSTM_CRF(hparams, attr_dict['tag2idx']).to(hparams['device'])

In [ ]:
base_model = train(hparams, base_model, loaders)

In [ ]:
evaluate(base_model, loaders[1])

In [ ]:
torch.save(base_model.state_dict(), hparams['save_path'])

In [ ]:
# base_model.load_state_dict(torch.load(hparams['save_path']))

In [ ]:
predict(['中国人民大学第三十五届一二九合唱音乐节如期举行，信息学院分团委文化部将组织信院全体同学参加','张配天获得最佳论文奖'], base_model, attr_dict['tokenizer'], 100)